<a href="https://colab.research.google.com/github/satyajeetsen007/Classification_of-_Loan_Borrowers_Kaggle/blob/main/Classification_of_Loan_Borrowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd
import pyxlsb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

In [ ]:
!pip install pyxlsb
!pip install catboost

In [7]:
train = pd.read_excel("training.xlsb", engine='pyxlsb')
test = pd.read_excel("test.xlsb", engine='pyxlsb')

In [8]:
train = train.iloc[:89734,:]
test = test.iloc[:38405,:]

In [ ]:
pd.set_option('display.max_columns',None)
train.describe(include='all')

In [ ]:
train.dtypes

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
### Missing values
print("Missing Values in Training df: ",train.isna().sum().sum())
print("Missing Values in Test df: ",test.isna().sum().sum())

In [ ]:
### Categorical Values
cat_var = [cols for cols in train.columns if train[cols].dtype == 'O']
print("No. of categorical values :", len(cat_var) ,'\n', cat_var, '\n')
### Cardinality
for cols in cat_var:
    print("Column Name: ",cols, "Unique Values: ",train[cols].unique(), "Count of Unique Values: ", len(train[cols].unique()),
          "Count : ", train[cols].value_counts(), "Percentage : ", train[cols].value_counts()/len(train[cols]),
          sep = '\n\n', end = '\n\n')
    print('-'*50)

In [14]:
### Rename the Known Columns
new_cols = {'I':'Gender','K':'Region', 'M':'Job_Title', 'N':'Education', 'O':'Marital_Status',
            'P':'Children', 'Q':'Property', 'S':'Employment_Status'}

In [15]:
train.rename(columns = new_cols, inplace=True)
test.rename(columns = new_cols, inplace=True)

In [ ]:
### Numerical Variables
num_var = [cols for cols in train.columns if train[cols].dtype != 'O']
print("No. of numerical values :", len(num_var) ,'\n', num_var, '\n')
### Distribution
sns.pairplot(train)
plt.show()

In [ ]:
correlation = train.corr()
correlation.style.background_gradient(cmap='coolwarm')

In [ ]:
for cols in num_var:
    sns.boxplot(train[cols])
    plt.title(cols)
    plt.show()

In [ ]:
### Imbalance Data??
print(train['MARKER'].value_counts())
print(test['MARKER'].value_counts())

In [20]:
### One-HOT Encoding
X = pd.get_dummies(train, drop_first=True)
y = pd.get_dummies(test, drop_first=True)

In [21]:
### Removing Columns to tackle Multi Collinearity (Corr > 0.6)
X = X.drop(['A','D','F'],axis =1)
y = y.drop(['A','D','F'],axis =1)

In [ ]:
correlation = X.corr()
correlation.style.background_gradient(cmap='coolwarm')

In [23]:
### Splitting the data - X,y - Train and Test
X_train = X.drop(['ID','MARKER'], axis =1)
y_train = X['MARKER']
X_test = y.drop(['ID','MARKER'], axis =1)
y_test = y['MARKER']

In [24]:
### Scaling the data
scaler = MinMaxScaler()
scaler.fit(X_train)

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

In [ ]:
print(len(X_train))
### Imbalance Data treatment
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
print(len(X_train_resampled))
#print(X_train_resampled.head())

In [ ]:
### Feature Selection
#sel_from_model = SelectFromModel(Lasso(alpha=0.1,random_state=12))
#sel_from_model.fit(X_train_resampled,y_train_resampled)

In [ ]:
'''selected_feat = X_train_resampled.columns[(sel_from_model.get_support())]

# let's print some stats
print('selected features: {}'.format(selected_feat))
print('total features: {}'.format((X_train_resampled.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(
    np.sum(sel_from_model.estimator_.coef_ == 0)))'''

In [ ]:
### Trying different data transformations

colss = ([x for x in X_train_resampled.columns if X_train_resampled[x].dtype == float])
print(X_train_resampled[colss].head())
X_train_resampled[colss] = np.absolute(X_train_resampled[colss])
print(X_train_resampled.head())
#'''print(X_train_resampled[colss].head())
#X_train_resampled[colss] = X_train_resampled[colss]**2
#print(X_train_resampled[colss].head())'''

In [30]:
#### Applying Logistic Reg
from  sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve, precision_score,recall_score,f1_score,classification_report

In [ ]:
params = {"penalty": ["l1", "l2", "elasticnet", "none"],"C": [1.0, 1.5, 0.5],
          "solver": ["newton-cg", "lbfgs", "liblinear", "sag", "saga"],"max_iter": [100, 150, 180]}

log_model = LogisticRegression()
model1 = GridSearchCV(estimator=log_model, param_grid=params,scoring = "recall", refit = True,verbose = 4, cv = 2)
model1.fit(X_train_resampled,y_train_resampled)

In [53]:
model1.best_estimator_

LogisticRegression(C=1.5)

In [ ]:
model2 = GridSearchCV(estimator=log_model, param_grid=params,scoring = "precision", refit = True,verbose = 4, cv = 2)
model2.fit(X_train_resampled,y_train_resampled)

In [55]:
model2.best_estimator_

LogisticRegression(max_iter=180, penalty='none')

In [57]:
model3 = GridSearchCV(estimator=log_model, param_grid=params,scoring = "f1", refit = True,verbose = 4, cv = 2)
model3.fit(X_train_resampled,y_train_resampled)

Fitting 2 folds for each of 180 candidates, totalling 360 fits
[CV 1/2] END C=1.0, max_iter=100, penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 2/2] END C=1.0, max_iter=100, penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 1/2] END C=1.0, max_iter=100, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/2] END C=1.0, max_iter=100, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/2] END C=1.0, max_iter=100, penalty=l1, solver=liblinear;, score=0.794 total time=  10.0s
[CV 2/2] END C=1.0, max_iter=100, penalty=l1, solver=liblinear;, score=0.846 total time=  11.8s
[CV 1/2] END C=1.0, max_iter=100, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 2/2] END C=1.0, max_iter=100, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 1/2] END C=1.0, max_iter=100, penalty=l1, solver=saga;, score=0.794 total time=   7.5s
[CV 2/2] END C=1.0, max_iter=100, penalty=l1, solver=saga;, score=0.846 total time=   5.9s
[CV 1/2] END C=1.0,

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 1/2] END C=1.0, max_iter=100, penalty=l2, solver=lbfgs;, score=0.794 total time=   1.7s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 2/2] END C=1.0, max_iter=100, penalty=l2, solver=lbfgs;, score=0.845 total time=   1.7s
[CV 1/2] END C=1.0, max_iter=100, penalty=l2, solver=liblinear;, score=0.794 total time=   1.4s
[CV 2/2] END C=1.0, max_iter=100, penalty=l2, solver=liblinear;, score=0.845 total time=   1.7s
[CV 1/2] END C=1.0, max_iter=100, penalty=l2, solver=sag;, score=0.794 total time=   2.5s
[CV 2/2] END C=1.0, max_iter=100, penalty=l2, solver=sag;, score=0.845 total time=   2.3s
[CV 1/2] END C=1.0, max_iter=100, penalty=l2, solver=saga;, score=0.794 total time=   2.4s
[CV 2/2] END C=1.0, max_iter=100, penalty=l2, solver=saga;, score=0.845 total time=   2.4s
[CV 1/2] END C=1.0, max_iter=100, penalty=elasticnet, solver=newton-cg;, score=nan total time=   0.0s
[CV 2/2] END C=1.0, max_iter=100, penalty=elasticnet, solver=newton-cg;, score=nan total time=   0.0s
[CV 1/2] END C=1.0, max_iter=100, penalty=elasticnet, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/2] END C=1.0, max_iter=100, penalty=elasticnet

...

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


[CV 1/2] END C=0.5, max_iter=180, penalty=none, solver=saga;, score=0.794 total time=  18.1s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
162 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
18 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solve

[CV 2/2] END C=0.5, max_iter=180, penalty=none, solver=saga;, score=0.846 total time=  18.0s


GridSearchCV(cv=2, estimator=LogisticRegression(),
             param_grid={'C': [1.0, 1.5, 0.5], 'max_iter': [100, 150, 180],
                         'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             scoring='f1', verbose=4)

In [58]:
model3.best_estimator_

LogisticRegression(penalty='none', solver='newton-cg')

In [ ]:
model1_be = LogisticRegression(C=1.5)
model1_be.fit(X_train_resampled,y_train_resampled)

In [ ]:
model2_be = LogisticRegression(max_iter=180, penalty='none')
model2_be.fit(X_train_resampled,y_train_resampled)

In [ ]:
model3_be = LogisticRegression(penalty='none', solver='newton-cg')
model3_be.fit(X_train_resampled,y_train_resampled)

In [60]:
### Results of M1 model

y_pred_m1 = model1_be.predict(X_test)
y_train_predict_m1 = model1_be.predict(X_train_resampled)
print("--"*5,"Results of M1 model","--"*5)
print("Accuracy Score of Model1_train : ", accuracy_score(y_train_resampled,y_train_predict_m1))
print("Accuracy Score of Model1_test : ", accuracy_score(y_test,y_pred_m1))
print('--'*5,"Training Data",'--'*5)
print(classification_report(y_test, y_pred_m1))
print(confusion_matrix(y_test, y_pred_m1))
print('--'*5,"Test Data",'--'*5)
print(classification_report(y_train_resampled, y_train_predict_m1))
print(confusion_matrix(y_train_resampled, y_train_predict_m1))

---------- Results of M1 model ----------
Accuracy Score of Model1_train :  0.8365681492751029
Accuracy Score of Model1_test :  0.5601353990365838
---------- Training Data ----------
              precision    recall  f1-score   support

           0       1.00      0.56      0.72     38259
           1       0.01      0.61      0.01       146

    accuracy                           0.56     38405
   macro avg       0.50      0.58      0.36     38405
weighted avg       0.99      0.56      0.71     38405

[[21423 16836]
 [   57    89]]
---------- Test Data ----------
              precision    recall  f1-score   support

           0       0.85      0.81      0.83     89392
           1       0.82      0.86      0.84     89392

    accuracy                           0.84    178784
   macro avg       0.84      0.84      0.84    178784
weighted avg       0.84      0.84      0.84    178784

[[72756 16636]
 [12583 76809]]


In [48]:
### Results of M2 model

y_pred_m2 = model2_be.predict(X_test)
y_train_predict_m2 = model2_be.predict(X_train_resampled)
print("--"*5,"Results of M2 model","--"*5)
print("Accuracy Score of Model2_train : ", accuracy_score(y_train_resampled,y_train_predict_m2))
print("Accuracy Score of Model2_test : ", accuracy_score(y_test,y_pred_m2))
print('--'*5,"Training Data",'--'*5)
print(classification_report(y_test, y_pred_m2))
print(confusion_matrix(y_test, y_pred_m2))
print('--'*5,"Test Data",'--'*5)
print(classification_report(y_train_resampled, y_train_predict_m2))
print(confusion_matrix(y_train_resampled, y_train_predict_m2))

---------- Results of M2 model ----------
Accuracy Score of Model2_train :  0.8368534097010918
Accuracy Score of Model2_test :  0.5601093607603177
---------- Training Data ----------
              precision    recall  f1-score   support

           0       1.00      0.56      0.72     38259
           1       0.01      0.62      0.01       146

    accuracy                           0.56     38405
   macro avg       0.50      0.59      0.36     38405
weighted avg       0.99      0.56      0.71     38405

[[21421 16838]
 [   56    90]]
---------- Test Data ----------
              precision    recall  f1-score   support

           0       0.85      0.81      0.83     89392
           1       0.82      0.86      0.84     89392

    accuracy                           0.84    178784
   macro avg       0.84      0.84      0.84    178784
weighted avg       0.84      0.84      0.84    178784

[[72783 16609]
 [12559 76833]]


In [62]:
### Results of M3 model

y_pred_m3 = model3_be.predict(X_test)
y_train_predict_m3 = model3_be.predict(X_train_resampled)
print("--"*5,"Results of M3 model","--"*5)
print("Accuracy Score of Model3_train : ", accuracy_score(y_train_resampled,y_train_predict_m3))
print("Accuracy Score of Model3_test : ", accuracy_score(y_test,y_pred_m3))
print('--'*5,"Training Data",'--'*5)
print(classification_report(y_test, y_pred_m3))
print(confusion_matrix(y_test, y_pred_m3))
print('--'*5,"Test Data",'--'*5)
print(classification_report(y_train_resampled, y_train_predict_m3))
print(confusion_matrix(y_train_resampled, y_train_predict_m3))

---------- Results of M3 model ----------
Accuracy Score of Model3_train :  0.836875783067836
Accuracy Score of Model3_test :  0.559953131102721
---------- Training Data ----------
              precision    recall  f1-score   support

           0       1.00      0.56      0.72     38259
           1       0.01      0.62      0.01       146

    accuracy                           0.56     38405
   macro avg       0.50      0.59      0.36     38405
weighted avg       0.99      0.56      0.71     38405

[[21415 16844]
 [   56    90]]
---------- Test Data ----------
              precision    recall  f1-score   support

           0       0.85      0.81      0.83     89392
           1       0.82      0.86      0.84     89392

    accuracy                           0.84    178784
   macro avg       0.84      0.84      0.84    178784
weighted avg       0.84      0.84      0.84    178784

[[72784 16608]
 [12556 76836]]


In [ ]:
#### Applying SVM
from  sklearn import svm
model4 = svm.SVC(kernel="poly", degree =1, decision_function_shape='ovo',gamma = 'auto')
model4.fit(X_train_resampled,y_train_resampled)

In [ ]:
### Results of M4 model

y_pred_m4 = model4.predict(X_test)
y_train_predict_m4 = model4.predict(X_train_resampled)
print("--"*5,"Results of M4 model","--"*5)
print("Accuracy Score of Model4_train : ", accuracy_score(y_train_resampled,y_train_predict_m4))
print("Accuracy Score of Model4_test : ", accuracy_score(y_test,y_pred_m4))
print('--'*5,"Training Data",'--'*5)
print(classification_report(y_test, y_pred_m4))
print(confusion_matrix(y_test, y_pred_m4))
print('--'*5,"Test Data",'--'*5)
print(classification_report(y_train_resampled, y_train_predict_m4))
print(confusion_matrix(y_train_resampled, y_train_predict_m4))

In [ ]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(model1,X_test,y_test,cv=10)

In [ ]:
score

In [ ]:
## Apply RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
model5 = RandomForestClassifier()
model5.fit(X_train_resampled, y_train_resampled)

In [ ]:
### Results of M5 model

y_pred_m5 = model5.predict(X_test)
y_train_predict_m5 = model5.predict(X_train_resampled)
print("--"*5,"Results of M5 model","--"*5)
print("Accuracy Score of Model5_train : ", accuracy_score(y_train_resampled,y_train_predict_m5))
print("Accuracy Score of Model5_test : ", accuracy_score(y_test,y_pred_m5))
print('--'*5,"Training Data",'--'*5)
print(classification_report(y_test, y_pred_m5))
print(confusion_matrix(y_test, y_pred_m5))
print('--'*5,"Test Data",'--'*5)
print(classification_report(y_train_resampled, y_train_predict_m5))
print(confusion_matrix(y_train_resampled, y_train_predict_m5))

In [ ]:
## Apply LGBM Classifier
lgb_model = LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42, n_jobs=1)
lgb_model.fit(X_train_resampled, y_train_resampled)

In [ ]:
y_pred = lgb_model.predict(X_test)
y_train_predict = lgb_model.predict(X_train_resampled)

print("Accuracy Score of Model1_train : ", accuracy_score(y_train_resampled,y_train_predict))
print("Accuracy Score of Model1_test : ", accuracy_score(y_test,y_pred))

In [ ]:
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print('--'*50)
print(classification_report(y_train_resampled, y_train_predict))
print(confusion_matrix(y_train_resampled, y_train_predict))

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada_classifier=AdaBoostClassifier()
ada_classifier.fit(X_train_resampled, y_train_resampled)

In [ ]:
y_pred = ada_classifier.predict(X_test)
y_train_predict = ada_classifier.predict(X_train_resampled)

In [ ]:
print("Accuracy Score of Model1_train : ", accuracy_score(y_train_resampled,y_train_predict))
print("Accuracy Score of Model1_test : ", accuracy_score(y_test,y_pred))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_classifier=KNeighborsClassifier()
knn_classifier.fit(X_train, y_train)

In [ ]:
y_pred = knn_classifier.predict(X_test)
y_train_predict = knn_classifier.predict(X_train_resampled)

In [ ]:
print("Accuracy Score of Model1_train : ", accuracy_score(y_train_resampled,y_train_predict))
print("Accuracy Score of Model1_test : ", accuracy_score(y_test,y_pred))

In [ ]:
import xgboost
xgb_m=xgboost.XGBClassifier()
xgb_m.fit(X_train_resampled, y_train_resampled)

In [ ]:
y_pred = xgb_m.predict(X_test)
y_train_predict = xgb_m.predict(X_train_resampled)

In [ ]:
print("Accuracy Score of Model1_train : ", accuracy_score(y_train_resampled,y_train_predict))
print("Accuracy Score of Model1_test : ", accuracy_score(y_test,y_pred))

In [ ]:
params={
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
}

In [ ]:
## Hyperparameter optimization using RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV
random_search=RandomizedSearchCV(xgb_m,param_distributions=params,n_iter=5,scoring='accuracy',n_jobs=-1,cv=5,verbose=3)
random_search.fit(X_train_resampled,y_train_resampled)

In [ ]:
y_pred = random_search.predict(X_test)
y_train_predict = random_search.predict(X_train_resampled)

print("Accuracy Score of Model1_train : ", accuracy_score(y_train_resampled,y_train_predict))
print("Accuracy Score of Model1_test : ", accuracy_score(y_test,y_pred))

In [ ]:
base_models = [
    ('catboost', CatBoostClassifier(
        iterations=100,
        learning_rate=0.1,
        depth=6,
        random_state=42
    )),
    ('xgboost', xgboost.XGBClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=6,
        random_state=42
    )),
    ('lightgbm', LGBMClassifier(
        n_estimators=100,
        learning_rate=0.1,
        random_state=42,
        n_jobs =1
    )),
    ('Logistic',LogisticRegression(
        penalty='none', solver='newton-cg'
    ))
]

In [ ]:
meta_model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    depth=6,
    random_state=42
)

In [ ]:
from sklearn.ensemble import StackingClassifier
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model)

In [ ]:
stacking_model.fit(X_train_resampled, y_train_resampled)

0:	learn: 0.5617382	total: 51ms	remaining: 5.05s
1:	learn: 0.4666516	total: 99.4ms	remaining: 4.87s
2:	learn: 0.4019319	total: 143ms	remaining: 4.63s
3:	learn: 0.3571307	total: 185ms	remaining: 4.44s
4:	learn: 0.3224676	total: 229ms	remaining: 4.34s
5:	learn: 0.2969781	total: 279ms	remaining: 4.38s
6:	learn: 0.2755221	total: 355ms	remaining: 4.72s
7:	learn: 0.2479807	total: 403ms	remaining: 4.63s
8:	learn: 0.2309756	total: 444ms	remaining: 4.49s
9:	learn: 0.2187999	total: 488ms	remaining: 4.4s
10:	learn: 0.2032517	total: 529ms	remaining: 4.28s
11:	learn: 0.1930926	total: 568ms	remaining: 4.17s
12:	learn: 0.1834596	total: 606ms	remaining: 4.05s
13:	learn: 0.1745252	total: 638ms	remaining: 3.92s
14:	learn: 0.1677816	total: 695ms	remaining: 3.94s
15:	learn: 0.1609010	total: 754ms	remaining: 3.96s
16:	learn: 0.1547764	total: 800ms	remaining: 3.9s
17:	learn: 0.1503973	total: 838ms	remaining: 3.82s
18:	learn: 0.1456393	total: 878ms	remaining: 3.74s
19:	learn: 0.1419347	total: 927ms	remaining

C:\Users\satya_omijkxl\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:45:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 89392, number of negative: 89392
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.073763 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1851
[LightGBM] [Info] Number of data points in the train set: 178784, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


C:\Users\satya_omijkxl\anaconda3\lib\site-packages\scipy\optimize\linesearch.py:477: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\satya_omijkxl\anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\satya_omijkxl\anaconda3\lib\site-packages\scipy\optimize\linesearch.py:477: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\satya_omijkxl\anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\satya_omijkxl\anaconda3\lib\site-packages\scipy\optimize\linesearch.py:477: LineSearchWarning: The line search algo

0:	learn: 0.5263964	total: 52.4ms	remaining: 5.19s
1:	learn: 0.4169071	total: 89.5ms	remaining: 4.39s
2:	learn: 0.3465447	total: 124ms	remaining: 4.02s
3:	learn: 0.2894939	total: 161ms	remaining: 3.87s
4:	learn: 0.2549382	total: 197ms	remaining: 3.75s
5:	learn: 0.2257771	total: 234ms	remaining: 3.66s
6:	learn: 0.2059680	total: 271ms	remaining: 3.6s
7:	learn: 0.1881374	total: 306ms	remaining: 3.52s
8:	learn: 0.1779084	total: 344ms	remaining: 3.48s
9:	learn: 0.1669918	total: 407ms	remaining: 3.67s
10:	learn: 0.1532261	total: 444ms	remaining: 3.59s
11:	learn: 0.1445467	total: 478ms	remaining: 3.51s
12:	learn: 0.1372710	total: 512ms	remaining: 3.43s
13:	learn: 0.1309692	total: 548ms	remaining: 3.37s
14:	learn: 0.1235238	total: 584ms	remaining: 3.31s
15:	learn: 0.1181587	total: 627ms	remaining: 3.29s
16:	learn: 0.1121372	total: 662ms	remaining: 3.23s
17:	learn: 0.1082778	total: 697ms	remaining: 3.17s
18:	learn: 0.1038058	total: 732ms	remaining: 3.12s
19:	learn: 0.0994076	total: 766ms	remain

...

C:\Users\satya_omijkxl\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:47:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:47:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:48:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:48:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

C:\Users\satya_omijkxl\anaconda3\lib\site-packages\scipy\optimize\linesearch.py:437: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
C:\Users\satya_omijkxl\anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\satya_omijkxl\anaconda3\lib\site-packages\sklearn\utils\optimize.py:204: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
C:\Users\satya_omijkxl\anaconda3\lib\site-packages\scipy\optimize\linesearch.py:477: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\satya_omijkxl\anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


0:	learn: 0.6045113	total: 23.5ms	remaining: 2.32s
1:	learn: 0.5309393	total: 44.4ms	remaining: 2.18s
2:	learn: 0.4702016	total: 64.9ms	remaining: 2.1s
3:	learn: 0.4176977	total: 86.6ms	remaining: 2.08s
4:	learn: 0.3734107	total: 108ms	remaining: 2.06s
5:	learn: 0.3352420	total: 129ms	remaining: 2.03s
6:	learn: 0.3024606	total: 149ms	remaining: 1.98s
7:	learn: 0.2732398	total: 173ms	remaining: 1.99s
8:	learn: 0.2475498	total: 194ms	remaining: 1.96s
9:	learn: 0.2252782	total: 214ms	remaining: 1.92s
10:	learn: 0.2054021	total: 237ms	remaining: 1.91s
11:	learn: 0.1878945	total: 259ms	remaining: 1.9s
12:	learn: 0.1723287	total: 279ms	remaining: 1.87s
13:	learn: 0.1587864	total: 300ms	remaining: 1.84s
14:	learn: 0.1463082	total: 320ms	remaining: 1.81s
15:	learn: 0.1356432	total: 341ms	remaining: 1.79s
16:	learn: 0.1259917	total: 365ms	remaining: 1.78s
17:	learn: 0.1173335	total: 386ms	remaining: 1.76s
18:	learn: 0.1097472	total: 407ms	remaining: 1.73s
19:	learn: 0.1028233	total: 431ms	remai

StackingClassifier(estimators=[('catboost',
                                <catboost.core.CatBoostClassifier object at 0x000001F4599E9130>),
                               ('xgboost',
                                XGBClassifier(base_score=None, booster=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              enable_categorical=False,
                                              gamma=None, gpu_id=None,
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learning_rate=0.1,
                                              max_delt...
                                              predictor=None, random_state=42,
                                         

In [ ]:
y_pred = stacking_model.predict(X_test)
y_train_predict = stacking_model.predict(X_train_resampled)

print("Accuracy Score of Model1_train : ", accuracy_score(y_train_resampled,y_train_predict))
print("Accuracy Score of Model1_test : ", accuracy_score(y_test,y_pred))

In [ ]:
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print('--'*50)
print(classification_report(y_train_resampled, y_train_predict))
print(confusion_matrix(y_train_resampled, y_train_predict))

In [ ]:
result = stacking_model.predict(X_test)
print(f'{sum(result)} defaults')

66.0 defaults


In [ ]:
results = pd.DataFrame()
results['ID'] = test.ID
results['My prediction'] = result
results['Real result'] = y_test
results.head()

In [ ]:
share_of_wrong_predicted = results[['My prediction', 'Real result']].duplicated().sum()
print(f'{share_of_wrong_predicted} from {results.shape[0]} observations were predicted right')

38401 from 38405 observations were predicted right


In [ ]:
results.to_csv('res.csv')